In [2]:
addprocs(11)

11-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12

In [3]:
# import FunctionalCollections
push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")
import Iterators
import Pipe
import Compat
import JLD
import DataStructures
import DataStructuresExtended
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD
@everywhere using DataStructures
@everywhere using DataStructuresExtended

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end



In [4]:
@everywhere const START_MARKER1 = Symbol("**START1**")
@everywhere const START_MARKER2 = Symbol("**START2**")
@everywhere const END_MARKER1 = Symbol("**END1**")
@everywhere const END_MARKER2 = Symbol("**END2**")
@everywhere typealias S Symbol
@everywhere typealias State{T} Tuple{T,T}
@everywhere const START_NODE_INDEX = 1
@everywhere const END_NODE_INDEX = 2

In [5]:
function load_counts(filename)
    counts = Dict{Tuple{Symbol,Symbol,Symbol},Int}()
    open(filename, "r") do fh
        for line in eachline(fh)
            word1,word2,word3,occurrences = split(line)
            trigram = (Symbol(word1),Symbol(word2),Symbol(word3))
            counts[trigram] = parse(Int,occurrences)
        end
    end
    counts
end
#kn_lm = KneserNeyProbDist(ccs)

load_counts (generic function with 1 method)

In [6]:


@everywhere type KneserNeyProbDist{T}
    """
    Kneser-Ney estimate of a probability distribution. This is a version of
    back-off that counts how likely an n-gram is provided the n-1-gram had
    been seen in training. Extends the ProbDistI interface, requires a trigram
    FreqDist instance to train on. Optionally, a different from default discount
    value can be specified. The default discount is set to 0.75.
    #Adapted from: http://www.nltk.org/_modules/nltk/probability.html
    """
    
    trigrams :: Accumulator{Tuple{T,T,T},Int}
    bigrams  :: Accumulator{Tuple{T,T},Int}
    trigrams_contain :: Accumulator{T,Int}
    discount :: Float64
    wordtypes_after :: Accumulator{Tuple{T,T},Int}
    wordtypes_before :: Accumulator{Tuple{T,T},Int}
end

In [7]:
@everywhere function KneserNeyProbDist{T}(trigrams :: Dict{Tuple{T,T,T},Int}, discount=0.75)
    """
    :param trigrams: The trigram frequency distribution upon which to base
        the estimation
    :param discount: The discount applied when retrieving counts of
        trigrams
    """
    # helper dictionaries used to calculate probabilities
    trigrams_contain = counter(T)
    bigrams  = counter(Tuple{T,T})
    wordtypes_after = counter(Tuple{T,T})
    wordtypes_before = counter(Tuple{T,T})

    for ((w0, w1, w2),n) in trigrams
        push!(trigrams_contain, w1)
        push!(bigrams, (w0,w1), n)
        push!(wordtypes_after,(w0,w1))  
        push!(wordtypes_before,(w1,w2))
    end
    KneserNeyProbDist{T}(counter(trigrams), bigrams, trigrams_contain, discount, wordtypes_after, wordtypes_before)
end
    

@everywhere function prob{T}(self::KneserNeyProbDist{T}, w0::T,w1::T, w2::T)
    # if the sample trigram was seen during training
    if (w0, w1,w2) in keys(self.trigrams)
        @assert self.trigrams[(w0,w1,w2)]>self.discount
        (self.trigrams[(w0,w1,w2)] - self.discount)/self.bigrams[(w0, w1)]
    # else if the 'rougher' environment was seen during training
    elseif (w0,w1) in keys(self.bigrams) && (w1,w2) in keys(self.wordtypes_before)
        aftr = self.wordtypes_after[(w0, w1)]
        bfr = self.wordtypes_before[(w1, w2)]

        # the probability left over from alphas
        leftover_prob = aftr * self.discount  / self.bigrams[(w0, w1)]
        @assert leftover_prob>0
        # the beta (including normalization)
        beta = bfr / (self.trigrams_contain[w1] - aftr)
        @assert beta>0
        leftover_prob * beta
    # else the sample was completely unseen during training
    else
        0.0
    end
end

In [8]:
kn_lm = open(deserialize, "results/data/books/train_books_corpus.kn3gram.jsz", "r") 

In [ ]:
@spawn prob(kn_lm, symbol("'m"), :so, :fast)

In [9]:
import JuMP
import MathProgBase
#import GLPKMathProgInterface
import Gurobi
@everywhere using JuMP
@everywhere using MathProgBase
#@everywhere using GLPKMathProgInterface
@everywhere using Gurobi

In [10]:
@everywhere function get_subtours(x::Matrix{JuMP.Variable})
    """
    returns the a vector of sets of node indexes, each set is a subtour
    The First subtour returned is the nonconnected one -- the path
    """
    x_val = getValue(x)
    x_iis,x_jjs, _  = findnz(x_val .>= 1 - 1e-6) #It just has to be close to 1 to be true
    nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])

    subtours = IntSet[]
    ii = START_NODE_INDEX
    push!(subtours, IntSet(END_NODE_INDEX)) #The END Node is always in the same subtour as the start node
    while(true)
        while(true) #Cycle through current subtour
            
            push!(subtours[end],ii) 
            
            jj = nodes_chain[ii]
            println(ii," ", jj)
            delete!(nodes_chain,ii)
            if jj∈subtours[end] 
                break 
            end
            ii=jj
        end   

        if length(nodes_chain)>0
            ii = first(keys(nodes_chain)) #start new subtour
            push!(subtours,IntSet())
        else
            break
        end
    end
    subtours
end

@everywhere function get_coretours(x_links)
    """
    Core tour goes from start to end (inclusive)
    """
    nodes_chain = Dict([ii=>jj for (ii,jj) in x_links])

    coretour = IntSet()
    ii = START_NODE_INDEX
    while(ii!=END_NODE_INDEX)            
        push!(coretour,ii) 
        ii = nodes_chain[ii]
    end   
    push!(coretour, END_NODE_INDEX)
    coretour
end

@everywhere function get_hyperclass(subtour,nodes,node_indexes_for_1st)
    tour_hyper_class_nodes=IntSet()
    for ii in subtour
        w1,w2 = nodes[ii]
        #println(unordered_markers[w1])
        class_nodes = node_indexes_for_1st[w1]
        union!(tour_hyper_class_nodes,class_nodes) 
    end
    tour_hyper_class_nodes
end

@everywhere function get_sentence{T}(x_val::Matrix,unordered_markers::T,nodes::Vector{State{Int}})
    x_iis,x_jjs, _  = findnz(x_val.>1-1e-6)
    nodes_chain = Dict([ii=>jj for (ii,jj) in zip(x_iis,x_jjs)])
    node_index=nodes_chain[START_NODE_INDEX]
    node_index=nodes_chain[node_index] #Skip the first two as they are the start nodes
    sent=T()
    while(node_index!=END_NODE_INDEX)
        node = nodes[node_index]
        push!(sent, unordered_markers[node[1]])
        node_index=nodes_chain[node_index]
    end
    sent
end

In [ ]:
eg_corpus = @pipe ["name this 1922 novel about leopold bloom written by james joyce",#* " .",
    "ralph waldo emerson dismissed this poet as the jingle man and james russell lowell called him three-fifths genius and two-fifths sheer fudge",# * " .",
    "this is the basis of a comedy of manners first performed in 1892",#*" .",
    "in a third novel a sailor abandons the patna and meets marlow who in another novel meets kurtz in the congo",
    "thus she leaves her husband and child for aleksei vronsky but all ends sadly when she leaps in front of a train",
    "we looked out at the setting sun .",
    " i went to the kitchen .",
    "how are you doing ?"
    ] |>map(split,_) |> map(shuffle,_)

In [ ]:
test_bag=eg_corpus[end]

#test_bag = shuffle(corpus[1022]) #length 28
#test_bag = shuffle(corpus[1028]) #length 20  (Gurodi 4.2, GLTK : 2064.6 seconds)
#test_bag = shuffle(corpus[1122]) #length 19 
#test_bag = shuffle(corpus[1000]) #length 17 (Gurbodi: 5.8 sconds. GLTK:  569.9seconds)
#test_bag = shuffle(["he was the greatest of man , a hero to many ."|> split])
#est_bag = shuffle(["this is but a silly joke -- that could never happen !"|> split])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a","fine","comedy", "."])
#^length 9, Gurodi 1.0, GLTK 5.3
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy", "."])
#test_bag =  shuffle(["it", "is", "so", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "good", "."])
#test_bag =  ["no", "way", "."]
#test_bag =  shuffle(["no", "."])
#@time best_order(test_bag, lm, mem_limit=1000)
test_bag = map(symbol, test_bag)

In [11]:
@everywhere function prepare_model(unordered_words::Vector, lm::Function, silent=true)
    m=Model(solver=GurobiSolver(OutputFlag=silent? 0:1, Threads=1))

    unordered_markers = [START_MARKER1; START_MARKER2; END_MARKER1; END_MARKER2; unordered_words...]
    #Note that this lacks END_MARKER2
    @assert START_NODE_INDEX == 1
    @assert END_NODE_INDEX == 2

    nodes = State{Int}[] #Named by word index

    node_indexes_for_1st = Dict{Int, Vector{Int}}()
    node_indexes_for_2nd = Dict{Int, Vector{Int}}()

    function add_node!(ii,jj)
        push!(nodes, (ii,jj))

        node_indexes_for_i_1st = get!(()->Int[], node_indexes_for_1st, ii)
        push!(node_indexes_for_i_1st, length(nodes))

        node_indexes_for_j_2nd = get!(()->Int[], node_indexes_for_2nd, jj) 
        push!(node_indexes_for_j_2nd, length(nodes))
        #println("node:$(length(nodes)) |  $(unordered_markers[ii])($ii), $(unordered_markers[jj])($jj)")
    end

    add_node!(1, 2) #That is START_MARKER1-> START_MARKER2
    add_node!(3, 4) #That is END_MARKER1-> END_MARKER2

    for ii in 1:length(unordered_markers)
        wi = unordered_markers[ii]
        if wi∈(END_MARKER1,START_MARKER1) continue end  #Covered these

        for jj in 1:length(unordered_markers)
            if ii==jj continue end
            wj = unordered_markers[jj]
            if wj∉(START_MARKER1,START_MARKER2,END_MARKER2)
                #but wj can be  END_MARKER1
                add_node!(ii,jj)
            end
        end
    end

    @defVar(m, x[1:length(nodes), 1:length(nodes)], Bin)

    #If you enter a node you must also leave it
    for (cc, center_node) in enumerate(nodes)
        if cc ∉ (START_NODE_INDEX, END_NODE_INDEX) #the beginning and end done have this requiement
            #Everything that enters this node, must leave this node
            @addConstraint(m, sum{x[ii,cc], ii=1:length(nodes)} == sum{x[cc,jj], jj=1:length(nodes)})
        end
    end

    for class_index in 1:length(unordered_markers)
        w_class =  unordered_markers[class_index]
        if w_class∉(START_MARKER1,START_MARKER2)
            #Not rquired to make a transition so that START_MARKER1 or 2 ever occur in second position
            jjs = node_indexes_for_2nd[class_index]
            @addConstraint(m, sum{x[ii,jj],ii=1:length(nodes), jj=jjs}==1)
        end

        #The following constraint is not required, as if it shows uo in the second position other rules make it certain to show up in the first
        #if w_class∉(END_MARKER1,END_MARKER2)
        #    #Not rquired to make a transition so that END_MARKER1 or 2 ever occur in second position
        #    iis=node_indexes_for_1st[class_index]
        #    @addConstraint(m, sum{x[ii,jj], ii=iis,jj=1:length(nodes)}==1)
        #    rules[w_class*"*"]=Set(product(iis,1:length(nodes)))
        #end
    end

    log_trans_prob = spzeros(length(nodes), length(nodes))
    for (from_node_index, from_node) in enumerate(nodes)
        w1 = unordered_markers[from_node[1]]
        w2 = unordered_markers[from_node[2]]
        #If what was in the second state element does not end up in the first state element then it is not allowed.
        can_transition_to = Set(get(node_indexes_for_1st, from_node[2], Int[]))
            #You can transition to any node which has your second element as its first element
        for (to_node_index, to_node) in enumerate(nodes)
            if to_node_index in can_transition_to
                @assert(from_node[2]==to_node[1])
                w3= unordered_markers[to_node[2]]
                log_tp = log(lm(w1,w2,w3))
                if log_tp>-Inf
                    log_trans_prob[from_node_index, to_node_index] = log_tp
                    continue
                else
                    #Banned as prob zero transitions not allowed
                    @addConstraint(m, x[from_node_index,to_node_index]==0)
                end

            else
                #It is not a legal transition
                @addConstraint(m, x[from_node_index,to_node_index]==0)
            end
        end
    end
    
    function eleminate_subtours(cb)
        x_val = getValue(x)
        x_iis,x_jjs, _  = findnz(x_val .>= 1 - 1e-6) #It just has to be close to 1 to be true
        x_links=zip(x_iis,x_jjs)
        coretour = get_coretours(x_links)
        if length(coretour)==length(x_links)+1
            #no subtour to eleminate
            return
        end
        core_hyperclass = get_hyperclass(coretour, nodes, node_indexes_for_1st)

        #Also the First subtour must go to one of the other subtours 
        arcs_outof_coretour = AffExpr()
        for ii in core_hyperclass
            for jj in 1:length(nodes)
                if jj∉core_hyperclass
                    arcs_outof_coretour += x[ii,jj]
                end
            end
        end
        @addLazyConstraint(cb, arcs_outof_coretour >=1)
    end

    addLazyCallback(m, eleminate_subtours,fractional=false)

    #new Linear way using logprobs
    @setObjective(m, Max, sum{log_trans_prob[i,j]*x[i,j], i=1:length(nodes), j=1:length(nodes)})
    
    m,x, nodes,unordered_markers
end

In [12]:
@everywhere function lm(w0,w1,w2)
    prob(kn_lm,w0,w1,w2)
end
@everywhere function remote_lm(w0,w1,w2)
    remotecall_fetch(lm, 1, w0,w1,w2) #Ask processor 1 to tell us the prob
end

In [18]:
@everywhere function best_order(unordered_words::Vector, lm::Function, silent=true)
    tic()
    m,x,nodes,unordered_markers = prepare_model(unordered_words, lm, silent)

    status = solve(m,suppress_warnings=silent)
    time_to_solve = toq()
    solution_prob = e^getObjectiveValue(m)
    generated_order = if status==:Optimal
        x_val = getValue(x)
        get_sentence(x_val, unordered_markers, nodes)
    else
        unordered_words 
    end
    (generated_order, solution_prob, time_to_solve, status)
end


In [ ]:
#fetch(@spawnat 2 best_order([:he, :is, :the, :yo, :king, Symbol(".")], remote_lm))

In [13]:
corpus_name = "books_corpus_0.001_of_test_oracle"#"books_corpus_0.001_of_test_glove300"
test_set_blocks = load("results/bags/$(corpus_name).jld")

Dict{ByteString,Any} with 100 entries:
  "32" => [(ASCIIString["she","reached","up","and","followed","his","fingers","…
  "29" => [(ASCIIString["i","said",",","you","know","youre","going","to","get",…
  "1"  => [(ASCIIString["``","you","would","destroy","a","book","that","even","…
  "54" => [(ASCIIString["thorpe","rose",",","agitated","."],ASCIIString["thorpe…
  "78" => [(ASCIIString["dirk","remarks","as","i","swing","a","right","leg","an…
  "81" => [(ASCIIString["they","shook","hands",",","while","ron","turned","towa…
  "2"  => [(ASCIIString["the","representation","of","the","body","of","christ",…
  "74" => [(ASCIIString["but","the","enormous","interests","at","stake","here",…
  "41" => [(ASCIIString["this","meant","i","knew","that","my","people","did","n…
  "65" => [(ASCIIString["i","really","hoped","they","'d","be","on","their","bes…
  "51" => [(ASCIIString["i","stood",",","tightening","the","belt","of","my","ro…
  "53" => [(ASCIIString["her","hair","moved",",","and","he","could","s

In [14]:
using Blocks
using Lumberjack
add_truck(LumberjackTruck("ordering_$(corpus_name).log"), "file-logger")
function jldopen_append(func::Function, filename::AbstractString)
    mode = isfile(filename) ? "r+" : "w" #Only open with "w" if it does't already exist
    jldopen(func, filename, mode)
end

  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
Base.FS is deprecated.
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
Base.FS is deprecated.
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3
  likely near /home/ubuntu/.julia/v0.5/Blocks/src/Blocks.jl:3


jldopen_append (generic function with 1 method)

In [19]:
const res_type = Tuple{Array{Symbol,1},Array{Symbol,1},Float64,Float64,Symbol}

function result_log(ii, block_res)
    num_optimal=0
    num_infeasible=0
    num_noattempted=0
    for res in block_res
        status = res[end]
        if status==:Optimal
            num_optimal+=1
        elseif status==:Infeasible
            num_infeasible+=1
        elseif status==:NotAttempted
            num_noattempted+=1
        else
            error("Unexpected stats $(status)")
        end
    end

    Lumberjack.info("$ii done: Opt:$(num_optimal/length(block_res)*100)%, "*
                                "Infeas:$(num_infeasible/length(block_res)*100)%, "*
                                "NAttpt:$(num_noattempted/length(block_res)*100)%")
end

function run(test_set_blocks, save_path="ordering.jld")
    try
        Lumberjack.info("Began ordering")
        ii = 0 
        map(@pipe test_set_blocks |> keys |> map(key->parse(Int,key),_) |>  sort |> map(string,_)) do (ii)
            block = test_set_blocks[ii]

            block_res::Vector{res_type} = pmap(block,err_stop=true) do bow_data
                (ground_order_s, bow_s, selection_score) = bow_data
                bow=map(Symbol, bow_s)
                ground_order = map(Symbol,ground_order_s)
                if length(ground_order_s)<=18
                    #(ground_order, generated_order, solution_prob, time_to_solve, status)
                    res = best_order(bow, remote_lm)
                    (ground_order, res...)
                else
                    (ground_order, bow, NaN, 0.0, :NotAttempted)
                end
            end
            result_log(ii,block_res)
            
            jldopen_append(save_path) do fh
                write(fh,ii, block_res)
            end
            Lumberjack.debug("$ii written to disc")
        end
    catch err
        Lumberjack.error("Unhandled Error", base_exception=err)
    end
    Lumberjack.info("complete selection")
end


run (generic function with 2 methods)

In [20]:
run(test_set_blocks, "results/ordered/$(corpus_name).jld")

2016-02-03T20:02:52 - info: Began ordering
2016-02-03T20:03:46 - info: 1 done: Opt:70.1492537313433%, Infeas:1.4925373134328357%, NAttpt:28.35820895522388%


2016-02-03T20:03:48 - debug: 1 written to disc
2016-02-03T20:05:13 - info: 2 done: Opt:76.11940298507463%, Infeas:0.0%, NAttpt:23.88059701492537%
2016-02-03T20:05:13 - debug: 2 written to disc
2016-02-03T20:06:19 - info: 3 done: Opt:76.11940298507463%, Infeas:2.9850746268656714%, NAttpt:20.8955223880597%
2016-02-03T20:06:19 - debug: 3 written to disc
2016-02-03T20:08:18 - info: 4 done: Opt:85.07462686567165%, Infeas:1.4925373134328357%, NAttpt:13.432835820895523%
2016-02-03T20:08:18 - debug: 4 written to disc
2016-02-03T20:13:09 - info: 5 done: Opt:77.61194029850746%, Infeas:0.0%, NAttpt:22.388059701492537%
2016-02-03T20:13:09 - debug: 5 written to disc
2016-02-03T20:13:52 - info: 6 done: Opt:74.6268656716418%, Infeas:1.4925373134328357%, NAttpt:23.88059701492537%
2016-02-03T20:13:52 - debug: 6 written to disc
2016-02-03T20:14:49 - info: 7 done: Opt:82.08955223880598%, Infeas:1.4925373134328357%, NAttpt:16.417910447761194%
2016-02-03T20:14:49 - debug: 7 written to disc
2016-02-03T20:15

In [ ]:
sample = typeof(test_set_blocks)()
sample["1"]=test_set_blocks["1"][1:5]
sample["2"]=test_set_blocks["2"][1:5]
run(sample)

In [ ]:
load("failed_ordering.jld", 1 ) 

In [ ]:

function get_irreducible_inconsistent_subsystem(m::JuMP.Model)    
    grb_model = m.internalModel.inner
    num_constrs = Gurobi.num_constrs(grb_model)
    Gurobi.computeIIS(grb_model)
    iis_constrs = Gurobi.get_intattrarray(grb_model, "IISConstr",  1, num_constrs)
    m.linconstr[find(iis_constrs)]
end

In [ ]:
iis = get_irreducible_inconsistent_subsystem(m)

In [ ]:
count,constraint_index = indmax([length(constrant.terms.coeffs) for constrant in iis])
constraint = iis[constraint_index]

In [ ]:
tts

In [ ]:
m=Model(solver=GurobiSolver())
@defVar(m,x, Int)
@defVar(m,y, Int)
@addConstraint(m,y<=1000) #This one won't cause problems
@addConstraint(m,x>=6)    #This will
@addConstraint(m,y>=6)    #This will 
#@addConstraint(m,x+y<=11) #This will

status = solve(m)

#@assert status==:Infeasible
#get_irreducible_inconsistent_subsystem(m)

In [ ]:
getObjectiveV

In [ ]:
function get_prod(x, iis, jjs)
    net = 1.0
    for i in 1:size(x,1) 
        for j in 1:size(x,2)
            println(i,",", j, " =", x[i,j], " ", trans_prob[i,j])
            net*=max((x[i,j]-1)^2, trans_prob[i,j])
        end
    end
    net
end
values = falses(size(x))
values[1,3] = 1
values[3,6] = 1
values[6,7] = 1
println("----------\n")
get_prod(values, iis,jjs)


In [ ]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]
const log_lm = train_language_model(corpus, loglikelyhood=true)